In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_MODEL = "gpt-3.5-turbo"
TEMPERATURE = 0.7 # Value between 0 and 2

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

## The message structure

The way these models work is through a conversation interface, this conversation engine works through a series of messages in the format:

```json
{
    "role": "user",
    "content": "message"
}
```

As far as roles go, there are three:

 - "user"
 - "assistant"
 - "system"

In [ ]:
messages = [
    { "role": "user", "content": "Who are you?" }
]

In [ ]:
completions = client.chat.completions.create(
    model = OPENAI_MODEL,
    temperature = TEMPERATURE,
    messages = messages,
)

In [ ]:
type(completions)

In [ ]:
message_choice = completions.choices[0]
message_choice

In [ ]:
message_choice.message.content

## The `system` role

It is possible to use a *"system"* message to customise the assistant:

In [ ]:
messages = [
    {
        "role": "system", 
        "content": "You are Spaghetti Rigatoni, a high-cusine chef that is always preparing something tasty"
    },
    { "role": "user", "content": "Who are you?" }
]

In [ ]:
completions = client.chat.completions.create(
    model = OPENAI_MODEL,
    temperature = TEMPERATURE,
    messages = messages,
)

In [ ]:
print(completions.choices[0].message.content)

## A helper function

In [ ]:
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_MODEL = "gpt-3.5-turbo"
TEMPERATURE = 0.1 # Value between 0 and 2

client = OpenAI(api_key=OPENAI_API_KEY)

def get_response(prompt):
    messages = [
        { "role": "user", "content": prompt }
    ]
    completions = client.chat.completions.create(
        model = OPENAI_MODEL,
        temperature = TEMPERATURE,
        messages = messages,
    )
    return completions.choices[0].message.content
    

In [ ]:
response = get_response("Hello, my name is Antonio. Who are you?")

print(response)

## A summariser

In [ ]:
three_star_review = """
Nice quick read with lots of ideas. \
I don't think you could possibly remember half of their \
guidelines even on the third read-through. Maybe that's just me, though. \
Quite possibly the most entertaining code book, but that's more due to \
the chapter-heading cartoons (a few of which were surprisingly funny) \
more than the text itself.
One thing that was annoying was the authors' attempt to include examples \
from seemingly every language ever conceived. I knew the python, \
recognized some C, got totally lost in the javascript, and couldn't \
even identify some of the others. Would have been easier if they'd just \
picked a language and then supplemented others where necessary, instead of \
hopping around like a bee that can't pick a favorite flower.
Still though, some good ideas and my code will be cleaner as I \
move forward now that I've read it. Solid three stars in the end.
"""

### First attempt – as is

In [ ]:
prompt = "Summarize the text into a single sentence:"

In [ ]:
complete_prompt = prompt + " " + three_star_review
response = get_response(complete_prompt)

print(response)

### Second attempt - use delimiters

In [ ]:
prompt = """Summarize the text delimited by triple backticks into a single sentence.

```{text}```
"""

In [ ]:
complete_prompt = prompt.format(text=three_star_review)
print(complete_prompt)

In [ ]:
response = get_response(complete_prompt)

print(response)

### Third attempt - ask for structured output

In [ ]:
prompt = """Summarize the text delimited by triple backticks into a single sentence.
Provide the response as a JSON object with the following keys: summary

```{text}```
"""

In [ ]:
response = get_response(prompt.format(text=three_star_review))

print(response)

In [ ]:
import json

resp = json.loads(response)

resp['summary']

### Fourth attempt - check for actual reviews

In [ ]:
prompt = """You will be provided with text delimited by triple backticks. \
If it contains a review about a book, summarize the text.

If it does not contain a review about a book, simply write \"Not a book review\"

Provide the response as a JSON object with the following keys: summary, is_a_review

```{text}```
"""

In [ ]:
response = get_response(prompt.format(text=three_star_review))

print(response)

In [ ]:
not_a_review = """Boxes and ingredients are packed in facilities that handles Peanut, Nuts, Sesame, Fish, \
Crustaceans, Milk, Egg, Mustard, Celery, Soya, Gluten and Sulphites. Due to the war in Ukraine, it has been \
necessary to substitute sunflower oil with rapeseed oil in some products without a label change. The FSA have \
advised that allergic reactions to rapeseed oil are rare."""

In [ ]:
response = get_response(prompt.format(text=not_a_review))

print(response)

### Fifth attempt - add more tasks

In [ ]:
prompt = """You will be provided with text delimited by triple backticks. \
If it contains a review about a book perform the following actions:

1. Identify the sentiment – sentiment
2. Extract the keywords - keywords
3. Summarize the review - summary
4. Suggest a title for the review - title

If it does not contain a review about a book, simply write \"Not a book review\"

Provide the response as a JSON object with the following keys: summary, is_a_review, keywords, title, sentiment

```{text}```
"""

In [ ]:
response = get_response(prompt.format(text=three_star_review))

print(response)

In [ ]:
response = get_response(prompt.format(text=not_a_review))

print(response)

### Sixth attempt – "force it" to return JSON

#### Only available for `gpt-4-1106-preview`!

In [ ]:
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_MODEL = "gpt-4-1106-preview"
TEMPERATURE = 0.1 # Value between 0 and 2

client = OpenAI(api_key=OPENAI_API_KEY)

def get_response(prompt):
    messages = [
        { "role": "user", "content": prompt }
    ]
    completions = client.chat.completions.create(
        model = OPENAI_MODEL,
        temperature = TEMPERATURE,
        messages = messages,
        
        response_format={ "type": "json_object" }

    )
    return completions.choices[0].message.content
    

In [ ]:
response = get_response(prompt.format(text=three_star_review))

print(response)

In [ ]:
resp = json.loads(response)

print

In [ ]:
# response = get_response("Who are you?")

# print(response)

## Few-shot learning

In [ ]:
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_MODEL = "gpt-3.5-turbo"
TEMPERATURE = 0.1 # Value between 0 and 2

client = OpenAI(api_key=OPENAI_API_KEY)

def get_response(prompt):
    messages = [
        { "role": "user", "content": prompt }
    ]
    completions = client.chat.completions.create(
        model = OPENAI_MODEL,
        temperature = TEMPERATURE,
        messages = messages,
    )
    return completions.choices[0].message.content
    

In [ ]:
prompt = """I will give you a noun delimited by angle brackets, I want you to give a name for a specific instance of that noun.

>>>
{noun}
>>>
"""

In [ ]:
# aeiou
# 12345

In [ ]:
response = get_response(prompt.format(noun='Egg'))
print(response)

In [ ]:
response = get_response(prompt.format(noun='Train'))
print(response)

In [ ]:
prompt = """I will give you a noun delimited by angle brackets, I want you to give a name for a specific instance of that noun.

User: boat
Assistant: Boaty McBoatface

User: howitzer
Assistant: Cannon McCannonface

User: beer
Assistant: Lager McLagerface

>>>
{noun}
>>>
"""

In [ ]:
response = get_response(prompt.format(noun='Egg'))
print(response)

In [ ]:
response = get_response(prompt.format(noun='Train'))
print(response)

In [ ]:
response = get_response(prompt.format(noun='snow plow'))
print(response)

In [ ]:
response = get_response(prompt.format(noun='guimpe'))
print(response)

## Tokens?

Check out their documentation: https://platform.openai.com/tokenizer

In [ ]:
import tiktoken

In [ ]:
encoding = tiktoken.encoding_for_model(OPENAI_MODEL)

In [ ]:
tokens = encoding.encode("Hello, how are you doing?")
print(tokens)

In [ ]:
decoded = encoding.decode(tokens)
print(decoded)

In [ ]:
encoding.decode_single_token_bytes(9906)

In [ ]:
encoding.decode_single_token_bytes(9907)

In [ ]:
encoding.decode_single_token_bytes(9905)